In [ ]:
XH = [64]
ZL = 40
from SY import *
HX.LB(XH, ZL)

# P

  - Free Electrons

# Hallmarks of Cancer

  - Self-Sufficiency in Growth Signals

    - mitogenic growth signals (GS)
      - quiescent state → active proliferative state
      - ensure a proper behavior of the various cell types within a tissue.
      - transmitted
        - transmembrane receptors
      - signaling molecules
        - diffusible growth factors
        - extracellular matrix components
        - cell-to-cell adhesion/interaction molecules
    
      - oncogenes
        - mimicking GS

  normal cells in culture, proliferate
  appropriate diffusible mitogenic factors
  roper substratum
  for their integrins

  tumor cells
  reduced dependence
  exogenous growth stimulation
  generate their own growth signals

  - Deregulating Cellular Energetics and Avoiding Immune Destruction
  

# 64

  - 天
    - 天
      - 乾
    - 泽
      - 屯
    - 火
      - 同人
    - 雷
      - 无妄
    - 风
      - 讼
    - 水
      - 需
    - 山
      - 大畜
    - 地
      - 泰

  - 泽
    - 天
      - 夬
    - 泽
      - 兑
    - 火
      - 革
    - 雷
      - 随
    - 风
      - 益
    - 水
      - 困
    - 山
      - 咸
    - 地
      - 萃

  - 火
    - 天
      - 大有
    - 泽
      - 睽
    - 火
      - 离
    - 雷
      - 豫
    - 风
      - 家人
    - 水
      - 既济
    - 山
      - 贲
    - 地
      - 明夷

  - 雷
    - 天
      - 无妄
    - 泽
      - 随
    - 火
      - 豫
    - 雷
      - 震
    - 风
      - 益
    - 水
      - 解
    - 山
      - 小过
    - 地
      - 复

  - 风
    - 天
      - 讼
    - 泽
      - 益
    - 火
      - 家人
    - 雷
      - 益
    - 风
      - 巽
    - 水
      - 涣
    - 山
      - 渐
    - 地
      - 观

  - 水
    - 天
      - 需
    - 泽
      - 困
    - 火
      - 既济
    - 雷
      - 解
    - 风
      - 涣
    - 水
      - 坎
    - 山
      - 蹇
    - 地
      - 比

  - 山
    - 天
      - 大畜
    - 泽
      - 咸
    - 火
      - 贲
    - 雷
      - 小过
    - 风
      - 渐
    - 水
      - 蹇
    - 山
      - 艮
    - 地
      - 谦

  - 地
    - 天
      - 泰
    - 泽
      - 萃
    - 火
      - 明夷
    - 雷
      - 复
    - 风
      - 观
    - 水
      - 比
    - 山
      - 谦
    - 地
      - 坤


# 0 interview

  - VZV

## 50 HCC & CRLM

## 50.0 Acquisition / Processing

  - TCGA database 
    - RNA expression profiles
    - clinical data
    - via UCSC Xena platform

  - survival analysis
    - 368 samples
  
  - Processing
    - TPM format
    - log2-transformed

  - sc

    - sc dataset
      - GEO
      - 10 HCC
      - 9 CRLM

    - Quality control criteria
      - mitochondrial content
        - ＜ 20%
      - UMI counts
        - 200-20,000
      - gene counts
        - 200-5,000

    - Seurat functions 
      - NormalizeData
        - Data normalization
      - FindVariableFeatures
        - selection of highly variable genes
        - 2000 genes
      - ScaleData
        - data transformation
        - eliminating cell cycle effects with the parameters 
        - vars.to.regress = c("S.Score", "G2M.Score")

      - Dimensionality reduction methods 
        - UMAP
        - TSNE
        - Louvain

    - Batch effect correction
      - Harmony

    - Differential gene expression
      - Between
        - clusters
        - cell types
      - FindAllMarkers function
        - p-value < 0.05, log2FC > 0.25
        - expression proportion > 0.1

  - ST

    - HCC ST
      - 4 HCC samples
    - CRLM ST
      - 4 CRLM samples

    - SpaceRanger software
      - Post-quality control

    - SCTtransform algorithm
      - data normalization
      - selection of highly variable genes
      - data transformation

    - average number of spots
      - 4054
    - average UMI
      - 15317.4
    - gene counts
      - 3890.1
    - mitochondrial content
      - 4.5%

    - Seurat
      - Analysis
      - visualization

    - CARD algorithm
      - conditional autoregression based deconvolution
      - Deconvolution analysis

    - single-cell annotations
      - cell type predictions for each spot

    - CARD software
      - Visualization of cell types in ST

    - Signature scores
      - Seurat
        - GSVA package
        - AddModelScore

## 50.1 sc Expression Profile

  - Markers

    - epithelial cell
      - EPCAM
      - KRT18
      - KRT19
      - CDH1

    - fibroblast
      - DCN
      - THY1
      - COL1A1
      - COL1A2

    - endothelial cell
      - PECAM1
      - CLDN5
      - FLT1
      - RAMP2

    - T cell 
      - CD3D
      - CD3E
      - CD3G
      - TRAC

    - NK cell 
      - NKG7
      - GNLY
      - NCAM1
      - KLRD1

    - B cell
      - CD79A
      - IGHM
      - IGHG3
      - IGHA2

    - mast cell 
      - KIT
      - MS4A2
      - GATA2

  - Pearson correlation matrix
    - correlation between two continuous variables
      - clustering results
      - cell type classification
    - marker gene expression
    - heatmap

  - Marker gene expression
    - heatmaps
    - tSNE plots
    - confirming cell-type identification

  - Comparisons
    - cell type composition
    - patient origin
    - cell count
    - transcript detection
    - CNV status

## 50.2 ST

  - HCC enriched
    - LIHCT
    - tumor cells
  - CRLM enriched
    - LIHC
    - fibroblasts
    - endothelial cells
    - myeloid cells

  - CNV detection
    - InferCNV software
    - CNV score distribution

  - deconvolution analysis
    - CARD algorithm
    - based on single-cell data
    - cell type distribution

    
  - H&E staining images

  - tumor vs. non-tumor classification

    - dimensionality reduction
    - clustering analysis



    - tumor cells
    8 subgroups
    LIHCT1
    LIHCT3
    LIHCT4

    tSNE plot
    subgroup composition
    sample origin
    CNV status
    patient composition

    Marker genes for each subgroup were calculated
    expression heatmaps were created



    - non-tumor cells
    5 subgroups
    endothelial cells
    tumor cells
    myeloid cells

    LIHCT2 enrichment

    tSNE plot
    marker gene expression heatmaps

## 50.3 Subclassification Analysis of Tumor Cells

Tumor cells were isolated and subjected to dimensionality reduction and clustering analysis, resulting in 16 clusters. These clusters were then merged based on their marker gene functions, yielding 7 tumor cell subgroups (Figure 3A). A bar plot displays the cell composition of each sample, showing that HCC samples have a higher content of APOA2+NEO cells, while CRLM samples have a higher content of TFF3+NEO cells. Differential gene expression analysis was performed for each subgroup, and a heatmap of differentially expressed genes was generated (Figure 3B). GO Biological Process (GOBP) enrichment analysis was conducted for the marker genes of each subgroup, revealing their corresponding functions. We compared the CNV differences among different cell types from different sample origins, finding significant differences in CNV across most cell types (Figure 3C). Transcription factor analysis was also performed for each cell type, with the results visualized in a heatmap showing different transcription factors enriched in various cell types (Figure 3D).

Subsequent cell trajectory analysis revealed multiple paths leading to different endpoints, indicating the potential existence of different subtypes. The starting branches contained a large number of APOA2+NEO cells and tumor cells from HCC (Figure 3E). From the pseudotime analysis, APOA2+NEO cells were primarily at the starting points, while most other cells were at the endpoints. Genes related to pseudotime were identified, and their expression heatmap was generated (Figure 3F). Scatter plots of pseudotime for six selected tumor marker genes (APOA2, HIST1H2AG, HMGB2, IGKV1-5, LCN2, TFF3) were created (Figure 3G).

## 50.3 Cell Annotation Analysis

  - tumor heterogeneity clustering
    - epithelial cells
    - immune cells
    - fibroblasts


## 50.4 Subgroup Analysis of Cell Populations

  - further distinction for subgroups
    - Immune cells
    - epithelial cells
    - fibroblasts

  - standard Seurat workflow
  - specific markers for each subgroup
  - UMAP plots

## 50.5 CNV Analysis of Single Cells

  - CNVs in tumor cell subgroups
    - InferCNV
    - references
      - fibroblasts
      - endothelial cells
    - identify malignant cells within subgroups

## 50.6 Pseudotime Analysis of Single Cells

  - Monocle2 software
    - pseudotime analysis
      - epithelial
      - macrophage
      - CAF cell

  - DDRTree
    - dimensionality reduction algorithm
    - parameters set to default
    - cell differentiation

## 50.7 Transcription Factor Analysis

  - SCENIC software
    - transcription factor analysis across cell subgroups
    - default parameters
      - RcisTarget
      - GRNBoost motif databases

  - RcisTarget package
    - overexpressed transcription factor binding motifs

  - AUCell package
    - scored the activity of each regulatory module for each cell type

## 50.8 Cell-Cell Communication Analysis

  - cell-cell communication
    - CellChat package
    - normalized gene expression matrix was imported

  - Data preprocessing
    - identifyOverExpressedGenes
    - identifyOverExpressedInteraction
    - ProjectData
    - default parameters

  - ligand-receptor interactions
    - computeCommunProb
    - filterCommunication
    - computeCommunProbPathway

  - cell communication network
    - aggregateNet function

## 50.9 Statistical Analysis

  - R 4.1.3
    - data processing
    - statistical analysis
    - plotting

  - Pearson correlation coefficient
    - correlation between two continuous variables

  - chi-square test
    - categorical variables

  - continuous variables
    - Wilcoxon rank-sum test
    - T-test

  - survminer package
    - determine optimal cutoff values

  - survival package
    - Cox regression
    - Kaplan-Meier analyses